# Implementation
Author: Eduardo Miguel Firvida Donestevez

In this tutorial, you will learn how to:
- Static deformation of IEA 15MW Offshore Reference Turbine 

In [1]:
import os
from pathlib import Path

CURRENT_FOLDER = Path().resolve()
DATA_FOLDER = os.path.join(CURRENT_FOLDER, "data")
OUTPUT_FOLDER = os.path.join(CURRENT_FOLDER, "output")


## Read pyNumad blade definition

The blade definition was taken from the PyNumad Original github repositories examples
here: https://github.com/sandialabs/pyNuMAD/blob/main/examples/example_data/IEA-15-240-RWT.yaml


In [2]:
bladeYaml = os.path.join(DATA_FOLDER, "IEA-15-240-RWT.yaml")

Create mesh object, and mesh solid FEM blade

In [3]:
from turbine_mesher.mesh import Mesh

mesh = Mesh(bladeYaml, element_size=0.5)

Updating station: 49 TE separation from 0.000648680513013211 to 0.001


In [4]:
mesh.solid_mesh()

Output()

Mesh statistics:
Total nodes: 31184
Total cells: 88086
Cell types: tetra
 -> tetra: 88086 Cells

### View mesh with PyVista

To plot the mesh we need to transform the MeshioMesh to pyVista.

In [5]:
# import pyvista as pv
# import numpy as np

# def meshio_to_pyvista(mesh):
#     points = mesh.points
#     cells = []

#     # Itera sobre las celdas de tipo tetraédrico
#     for cell_block in mesh.cells_dict.values():
#         for cell in cell_block:
#             cells.append([4] + cell.tolist())
                
#     cells = np.array(cells)
#     return pv.PolyData(points,cells)

# pv_mesh = meshio_to_pyvista(mesh.mesh)
# print(pv_mesh)
# plotter = pv.Plotter()
# plotter.add_mesh(pv_mesh, color="lightblue", show_edges=True)
# plotter.show()

Prepare mesh for DolFinX

In [6]:
from mpi4py import MPI

import basix.ufl
import dolfinx as dfx

import ufl

nodes = mesh.mesh.points
connectivity = [i for i in mesh.mesh.cells_dict.values()][0]
c_el = ufl.Mesh(basix.ufl.element("Lagrange", "tetrahedron", 1, shape=(nodes.shape[1],)))
domain = dfx.mesh.create_mesh(MPI.COMM_SELF, connectivity, nodes, c_el)


We then create the mesh, which will consist of hexahedral elements, along with the function space.
As we want a vector element with three compoenets, we add `(3, )` or `(domain.geometry.dim, )` to the element tuple to make it a triplet
However, we also could have used `basix.ufl`s functionality, creating a vector element `element = basix.ufl.element("Lagrange", domain.topology.cell_name(), 1, shape=(domain.geometry.dim,))`, and intitializing the function space as `V = dolfinx.fem.functionspace(domain, element)`.

In [7]:
V = dfx.fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))

## Boundary conditions
As we would like to clamp the boundary at $x=0$, we do this by using a marker function, which locate the facets where $x$ is close to zero by machine precision.

In [8]:
import numpy as np

def clamped_boundary(x):
    return np.isclose(x[2], 0)


fdim = domain.topology.dim - 1
dim = domain.topology.dim
boundary_facets = dfx.mesh.locate_entities_boundary(domain, fdim, clamped_boundary)

u_D = np.array([0, 0, 0], dtype=dfx.default_scalar_type)
bc = dfx.fem.dirichletbc(u_D, dfx.fem.locate_dofs_topological(V, fdim, boundary_facets), V)

As we want the traction $T$ over the remaining boundary to be $0$, we create a `dolfinx.Constant`

In [9]:
ds = ufl.Measure("ds", domain=domain)

## Variational formulation
We are now ready to create our variational formulation in close to mathematical syntax, as for the previous problems.

In [10]:
E = dfx.fem.Constant(domain, 210e3)
nu = dfx.fem.Constant(domain, 0.3)

lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / 2 / (1 + nu)


def epsilon(v):
    return ufl.sym(ufl.grad(v))


def sigma(v):
    return lmbda * ufl.tr(epsilon(v)) * ufl.Identity(dim) + 2 * mu * epsilon(v)

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

rho = 2e-3
g = 9.81
f = dfx.fem.Constant(domain, np.array([ -rho * g,0, 0]))

dx = ufl.Measure("dx", domain=domain)
a = ufl.inner(sigma(u), epsilon(v)) * dx
L = ufl.inner(f, v) * ds

```{note}
Note that we used `nabla_grad` and optionally `nabla_div` for the variational formulation, as oposed to our previous usage of
`div` and `grad`. This is because for scalar functions $\nabla u$ has a clear meaning
$\nabla u = \left(\frac{\partial u}{\partial x}, \frac{\partial u}{\partial y}, \frac{\partial u}{\partial z} \right)$.

However, if $u$ is vector valued, the meaning is less clear. Some sources define $\nabla u$ as a matrix with the elements $\frac{\partial u_j}{\partial x_i}$, while other  sources prefer
$\frac{\partial u_i}{\partial x_j}$. In DOLFINx `grad(u)` is defined as the matrix with elements $\frac{\partial u_i}{\partial x_j}$. However, as it is common in continuum mechanics to use the other definition, `ufl` supplies us with `nabla_grad` for this purpose.
```

## Solve the linear variational problem
As in the previous demos, we assemble the matrix and right hand side vector and use PETSc to solve our variational problem

In [11]:
from dolfinx.fem.petsc import LinearProblem

problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

## Visualization

As in the previous demos, we can either use Pyvista or Paraview for visualization. We start by using Pyvista. Instead of adding scalar values to the grid, we add vectors.

In [12]:
import pyvista as pv
p = pv.Plotter()
topology, cell_types, geometry = dfx.plot.vtk_mesh(V)
grid = pv.UnstructuredGrid(topology, cell_types, geometry)

grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
actor_0 = p.add_mesh(grid, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=1.5)
actor_1 = p.add_mesh(warped, show_edges=True)
p.show_axes()
p.show()

Widget(value='<iframe src="http://localhost:40297/index.html?ui=P_0x78e9afaa7dd0_0&reconnect=auto" class="pyvi…

We could also use Paraview for visualizing this.
As explained in previous sections, we save the solution with `XDMFFile`.
After opening the file `deformation.xdmf` in Paraview and pressing `Apply`, one can press the `Warp by vector button` ![Warp by vector](warp_by_vector.png) or go through the top menu (`Filters->Alphabetical->Warp by Vector`) and press `Apply`. We can also change the color of the deformed beam by changing the value in the color menu ![color](color.png) from `Solid Color` to `Deformation`.

In [17]:
with dfx.io.XDMFFile(domain.comm, "deformation.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    uh.name = "Deformation"
    xdmf.write_function(uh)
    
with dfx.io.VTKFile(domain.comm, "deformation.pvd", "w") as xdmf:
    xdmf.write_mesh(domain)
    uh.name = "Deformation"
    xdmf.write_function(uh)

## Stress computation
As soon as the displacement is computed, we can compute various stress measures. We will compute the von Mises stress defined as $\sigma_m=\sqrt{\frac{3}{2}s:s}$ where $s$ is the deviatoric stress tensor $s(u)=\sigma(u)-\frac{1}{3}\mathrm{tr}(\sigma(u))I$.

In [14]:
s = sigma(uh) - 1.0 / 3 * ufl.tr(sigma(uh)) * ufl.Identity(len(uh))
von_Mises = ufl.sqrt(3.0 / 2 * ufl.inner(s, s))

The `von_Mises` variable is now an expression that must be projected into an appropriate function space so that we can visualize it. As `uh` is a linear combination of first order piecewise continuous functions, the von Mises stresses will be a cell-wise constant function.

In [15]:
V_von_mises = dfx.fem.functionspace(domain, ("DG", 0))
stress_expr = dfx.fem.Expression(von_Mises, V_von_mises.element.interpolation_points())
stresses = dfx.fem.Function(V_von_mises)
stresses.interpolate(stress_expr)

In the previous sections, we have only visualized first order Lagrangian functions. However, the Von Mises stresses are piecewise constant on each cell. Therefore, we modify our plotting routine slightly. The first thing we notice is that we  now set values for each cell, which has a one to one correspondence with the degrees of freedom in the function space.

In [16]:
warped.cell_data["VonMises"] = stresses.x.petsc_vec.array
warped.set_active_scalars("VonMises")
p = pv.Plotter()
p.add_mesh(warped)
p.show_axes()
p.show()

Widget(value='<iframe src="http://localhost:40297/index.html?ui=P_0x78e99a28f170_1&reconnect=auto" class="pyvi…